In [19]:
import numpy as np
import pandas as pd
from numpy.linalg import eig
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [20]:
data = pd.read_csv('C:/Users/arund/OneDrive/Desktop/B.Tech/SEM_5/SM_2/cars.csv')
data

,Manufacture,Model,Sales,Resales,Type,Price,Engine_size,Horsepow,WheelBase,Width,Length,Curb_Wgt,Fuel_Cap,Mpg,Inslaes
0,Acura,Integra,16.919,16.360,0,21.50,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2.83
1,Acura,TL,39.384,19.875,0,28.40,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,3.67
2,Acura,CL,14.114,18.225,0,NaN,3.2,225.0,106.9,70.6,192.0,3.470,17.2,26.0,2.65
3,Acura,RL,8.588,29.725,0,42.00,3.5,210.0,114.6,71.4,196.6,3.850,18.0,22.0,2.15
4,Audi,A4,20.397,22.255,0,23.99,1.8,150.0,102.6,68.2,178.0,2.998,16.4,27.0,3.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Volvo,V40,3.545,NaN,0,24.40,1.9,160.0,100.5,67.6,176.6,3.042,15.8,25.0,1.27
153,Volvo,S70,15.245,NaN,0,27.50,2.4,168.0,104.9,69.3,185.9,3.208,17.9,25.0,2.72
154,Volvo,V70,17.531,NaN,0,28.80,2.4,168.0,104.9,69.3,186.2,3.259,17.9,25.0,2.86
155,Volvo,C70,3.493,NaN,0,45.50,2.3,236.0,104.9,71.5,185.7,3.601,18.5,23.0,1.25


In [21]:
data = data.drop(['Manufacture','Model','Resales','Inslaes'], axis=1)

In [22]:
col = ['Sales','Price', 'Engine_size', 'Horsepow', 'WheelBase', 'Width', 'Length', 'Curb_Wgt','Fuel_Cap', 'Mpg']
scaler=StandardScaler()

data[col]=pd.DataFrame(scaler.fit_transform(data[col]),columns=data[col].columns)

In [23]:
#data = data-data.mean()

In [24]:
data.dropna(inplace = True)

In [25]:
data.head()

,Sales,Type,Price,Engine_size,Horsepow,WheelBase,Width,Length,Curb_Wgt,Fuel_Cap,Mpg
0,-0.532042,0,-0.411789,-1.210889,-0.812988,-0.825439,-1.118929,-1.116140,-1.175923,-1.226164,0.973544
1,-0.200761,0,0.070551,0.133586,0.690949,0.080457,-0.247036,0.415009,0.221133,-0.194023,0.270767
3,-0.654896,0,1.021248,0.421687,0.425549,0.933837,0.072658,0.691363,0.750996,0.012406,-0.432010
4,-0.480754,0,-0.237727,-1.210889,-0.636055,-0.641634,-0.857361,-0.697875,-0.604689,-0.400451,0.739285
5,-0.504599,0,0.458519,-0.250550,0.248615,0.159230,1.438623,0.347788,0.291145,0.141423,-0.432010


In [26]:
def evaluate(components, eigenValues, eigenVectors, df):
    weightage = {}
    
    col = ['PC'+ str(i+1) for i in range(components.shape[1])]
    transform = pd.DataFrame(components, columns = col)
    display(transform)
    
    var = eigenValues/sum(eigenValues)
    for i in range(1,len(var)+1):
        if sum(var[:i])>=0.9:
            no_of_pc = i
            break
    print(f'\n{no_of_pc} components explains 90% or more variability')
    
    for i in range(no_of_pc):
        top_n = np.argsort( abs(eigenVectors[i]) )[::-1][:4] # shows top 4 affecting features
        weightage['PC'+str(i+1)] = [df.columns[j] + ' (' + str(round(abs(eigenVectors[i][j]),4))+') ' for j in top_n]
    
    impact = pd.DataFrame.from_dict(weightage, orient='index')
    impact['Variability (%)'] = var[:no_of_pc]
    impact['Cummulative Variability (%)'] = [sum(var[:i]) for i in range(1,no_of_pc+1)]
    display(impact)
    return transform, impact, no_of_pc

In [27]:
def PCA_lib(df):
    pca = PCA(df.shape[1])
    pca.fit(df)
    eigenVectors = pca.components_.T
    eigenValues = pca.explained_variance_
    components = pca.transform(df)
    return components, eigenValues, eigenVectors

In [28]:
def PCA_self(df):
    mean_cols = []
    for i in df.columns: 
        df[i+'-avg'+i] = df[i]-np.mean(df[i])
        mean_cols.append(i+'-avg'+i)
    
    cov = df[mean_cols].cov()
    
    eigenValues, eigenVectors = eig(cov)
    
    idx = eigenValues.argsort()[::-1]   #sorting in descending order
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    
    components = np.dot(df[mean_cols], eigenVectors)
    return components, eigenValues, eigenVectors, cov

In [29]:
components, eigenValues, eigenVectors = PCA_lib(data)

In [30]:
transform, impact, no_of_pc = evaluate(components, eigenValues, eigenVectors, data)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11
0,3.015074,-0.488703,0.021026,-0.207167,-0.177979,-0.052203,-0.252942,0.122965,-0.122493,0.017453,-0.014302
1,-0.267452,-0.308674,-0.599569,-0.266769,-0.254312,0.380395,0.200342,-0.071682,-0.058870,-0.415433,-0.169988
2,-1.484929,-0.524663,-0.354815,-0.684215,-0.734881,0.189390,-0.298647,-0.088371,-0.486470,0.308972,-0.154810
3,2.042483,-0.328679,0.312583,-0.331536,-0.420821,-0.275609,-0.021546,-0.102897,0.170363,-0.126855,-0.125313
4,-1.050271,-0.220222,-0.200494,-0.757847,0.526651,-0.803488,-0.654181,-0.318950,0.175019,-0.027251,-0.027584
...,...,...,...,...,...,...,...,...,...,...,...
147,1.993532,-0.772292,0.527597,-0.240368,-0.278803,0.067844,-0.237388,-0.266495,0.144142,-0.156953,-0.170445
148,0.863202,-0.346210,0.225479,-0.458787,-0.387554,-0.028234,0.015194,-0.178510,0.278475,0.029117,-0.083308
149,0.801689,-0.365293,0.210802,-0.429995,-0.444444,-0.069029,0.026365,-0.241533,0.231489,0.037566,-0.079632
150,-0.457899,-1.427802,-0.111752,-0.278106,-0.623048,-0.645878,-0.503592,-0.426055,0.217208,-0.500753,-0.067344



5 components explains 90% or more variability


,0,1,2,3,Variability (%),Cummulative Variability (%)
PC1,Engine_size (0.7721),Type (0.5445),Price (0.2446),Length (0.1437),0.572664,0.572664
PC2,Mpg (0.8221),Price (0.3444),Fuel_Cap (0.2696),Curb_Wgt (0.2465),0.186043,0.758707
PC3,Type (0.488),Horsepow (0.4779),WheelBase (0.3952),Fuel_Cap (0.3492),0.070209,0.828916
PC4,Fuel_Cap (0.4546),Width (0.4408),WheelBase (0.3809),Sales (0.3632),0.064694,0.893610
PC5,Fuel_Cap (0.7),Type (0.3722),Price (0.3641),Sales (0.3129),0.032877,0.926487


In [31]:
components, eigenValues, eigenVectors, cov = PCA_self(data)

In [32]:
transform, impact, no_of_pc = evaluate(components, eigenValues, eigenVectors, data)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11
0,3.015074,-0.488703,0.021026,-0.207167,0.177979,0.052203,-0.252942,0.122965,0.122493,-0.017453,-0.014302
1,-0.267452,-0.308674,-0.599569,-0.266769,0.254312,-0.380395,0.200342,-0.071682,0.058870,0.415433,-0.169988
2,-1.484929,-0.524663,-0.354815,-0.684215,0.734881,-0.189390,-0.298647,-0.088371,0.486470,-0.308972,-0.154810
3,2.042483,-0.328679,0.312583,-0.331536,0.420821,0.275609,-0.021546,-0.102897,-0.170363,0.126855,-0.125313
4,-1.050271,-0.220222,-0.200494,-0.757847,-0.526651,0.803488,-0.654181,-0.318950,-0.175019,0.027251,-0.027584
...,...,...,...,...,...,...,...,...,...,...,...
147,1.993532,-0.772292,0.527597,-0.240368,0.278803,-0.067844,-0.237388,-0.266495,-0.144142,0.156953,-0.170445
148,0.863202,-0.346210,0.225479,-0.458787,0.387554,0.028234,0.015194,-0.178510,-0.278475,-0.029117,-0.083308
149,0.801689,-0.365293,0.210802,-0.429995,0.444444,0.069029,0.026365,-0.241533,-0.231489,-0.037566,-0.079632
150,-0.457899,-1.427802,-0.111752,-0.278106,0.623048,0.645878,-0.503592,-0.426055,-0.217208,0.500753,-0.067344



5 components explains 90% or more variability


,0,1,2,3,Variability (%),Cummulative Variability (%)
PC1,Engine_size (0.7721),Type (0.5445),Price (0.2446),Length (0.1437),0.572664,0.572664
PC2,Mpg (0.8221),Price (0.3444),Fuel_Cap (0.2696),Curb_Wgt (0.2465),0.186043,0.758707
PC3,Type (0.488),Horsepow (0.4779),WheelBase (0.3952),Fuel_Cap (0.3492),0.070209,0.828916
PC4,Fuel_Cap (0.4546),Width (0.4408),WheelBase (0.3809),Sales (0.3632),0.064694,0.893610
PC5,Fuel_Cap (0.7),Type (0.3722),Price (0.3641),Sales (0.3129),0.032877,0.926487


ValueError: too many values to unpack (expected 2)